<a href="https://colab.research.google.com/github/N-nolwenn/SAM/blob/main/Copie_de_TP1_2023_index_fichier_ETU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAM: TP1 Accès aux données avec index 

Sujet pour étudiants

date de modification : 26/01/2023 16h

NOM: BOUCHOUCHI et PIGEON

Prénom: Nour et Nolwenn

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'un fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [ ]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

DATA = "data.csv"

# Générer un fichier

Création du fichier

In [ ]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 * 1000
# nb_lines = 100
nb_attributes = 7

longueur_attribut = 100
# string_val = "".join(choice(ascii_lowercase) for i in range(longueur_attribut))
long_string = ''.join('-' for i in range(longueur_attribut))

# a=[np.random.randint(0, int(nb_lines/(10**i)), nb_lines) for i in range(nb_attributes)]
nb_valeurs_distinctes = nb_lines

# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
  nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
  a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

b = [ ','.join(map(lambda x: str(x), e)) + f",{long_string}\n" for e in zip(*a)]

with open(DATA, "w") as f:
  f.write(''.join(b))

In [ ]:
%%bash
echo "head : "
head -n 2 data.csv
echo "tail : "
tail -n 2 data.csv
echo "size (lines) :"
wc -l data.csv

head : 
3374854,999478,661438,333402,17341,57062,35751,----------------------------------------------------------------------------------------------------
1871912,2333597,474092,48205,137169,153149,25428,----------------------------------------------------------------------------------------------------
tail : 
3830079,99941,846378,68098,260228,38712,9305,----------------------------------------------------------------------------------------------------
3176394,767655,265970,301335,252048,98988,19697,----------------------------------------------------------------------------------------------------
size (lines) :
5000000 data.csv


# Lecture séquentielle

In [ ]:
def filtrer_fichier(fichier, valeur_recherchee):
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      a = int(line.split(',')[0]) #cle unique
      if a == s :
        print(f"ligne {i} :", line.strip())


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier(DATA, s)
print("done in", time.time() - t1, "s")

valeur recherchée : 24953
ligne 997689 : 24953,1369506,937797,296988,84051,92800,38868,----------------------------------------------------------------------------------------------------
done in 4.316120624542236 s


# Découper le fichier en pages

In [ ]:
def page_dir_name(fichier):
  return fichier.split('.')[0] + "_pages"

def decoupe_fichier_en_pages(fichier, nb_tuple_par_page):
  page_dir = page_dir_name(fichier)
  print("pages dans :", page_dir)
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  with open(fichier, "r") as f:
    p=0
    lines = []
    for i, line in enumerate(f):
      lines.append(line)
      if (i+1) % nb_tuple_par_page == 0:
        p += 1
        with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
        lines = []
    if len(lines) > 0:
      p +=1
      with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
    
    print("nb pages créées :", p)

decoupe_fichier_en_pages(DATA, nb_tuple_par_page=1000)

pages dans : data_pages
nb pages créées : 5000


Afficher le nombre de tuples dans une page (pour quelques pages)

In [ ]:
%%bash
wc -l data_pages/* | head -n 3

     1000 data_pages/page1
     1000 data_pages/page10
     1000 data_pages/page100


# Lecture séquentielle du fichier découpé en pages

In [ ]:
def lecture_sequentielle_par_page(fichier):
   page_dir = page_dir_name(fichier)
   nb_pages = len(os.listdir(page_dir))
   
   # a faire : pour chaque page, lire ses lignes
   # une ligne devient un tuple
   # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
   for p in range(nb_pages) :
     with open(page_dir + f"/page{(p+1)}", "r") as fp:
       for i, line in enumerate(fp):
         t = line.split(',')
         yield p+1,i+1,t
   


def filtrer_fichier_par_pages(fichier, valeur_recherchee):
  # à faire pour chaque (numéro de page, position dans la page, tuple) obtenu en invoquant la méthode ci-dessus
  # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee  
  y = lecture_sequentielle_par_page(fichier)
  for line in y:
    a = int(line[2][0])
    if a == valeur_recherchee : 
      print(line)



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages("data.txt", s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 63206
(2981, 843, ['63206', '2484220', '717793', '242299', '301552', '114687', '69313', '----------------------------------------------------------------------------------------------------\n'])
done in 5.44 s


# Lecture d'un tuple dans une page

In [ ]:
def lecture_tuple(fichier, num_page, position):
  page_dir = page_dir_name(fichier)
  with open(page_dir + f"/page{num_page}", "r") as fp:
    for i, line in enumerate(fp):
      if(i==position-1):
        print(line)  
        return line


t1 = time.time()
lecture_tuple("data.txt", 188,6)
print("done in", round(time.time() - t1, 2), "s")

1008799,600545,375795,347887,145831,45618,64966,----------------------------------------------------------------------------------------------------

done in 0.0 s


# Créer un index

In [ ]:
def creation_index_unique(fichier):
  index = {}

  # la clé est la valeur du 1er attribut
  # la valeur est un rowid composé de (page, position)
  y = lecture_sequentielle_par_page(fichier)

  for line in y : 
    index[int(line[2][0])] = (line[0], line[1])
  return index

t1 = time.time()
index1 = creation_index_unique("data.txt")
print("done in", round(time.time() - t1, 2), "s")

done in 8.99 s


In [ ]:
def creation_index_not_unique(fichier):
  index = {}

  # la clé est la valeur du 1er attribut
  # la valeur est un rowid composé de (page, position)
  y = lecture_sequentielle_par_page(fichier)

  for line in y :
    if(int(line[2][1]) in index):
      index[int(line[2][1])] = index[int(line[2][1])]+[(line[0], line[1])]
    else : 
      index[int(line[2][1])] = [(line[0], line[1])]
  return index

t1 = time.time()
index2 = creation_index_not_unique("data.txt")
print("done in", round(time.time() - t1, 2), "s")

done in 64.09 s


In [ ]:
print(index1[3241396])

(188, 11)


In [ ]:
print(index2[2466832])

[(188, 11), (874, 45), (3108, 422), (3765, 596), (4008, 23)]


# Accès par index

## Index unique scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur donnée.

On peut supposer pour simplifier que l'attribut est unique

In [ ]:
def selection_par_index(fichier, index, valeur_recherchee):
  num_page, position = index[valeur_recherchee]
  return lecture_tuple(fichier, num_page, position)

s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
selection_par_index("data.txt", index1, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 70852
70852,2173908,38506,155950,113870,114453,27492,----------------------------------------------------------------------------------------------------

done in 0.0 s


## Index range scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur comprise dans une intervalle donné

In [ ]:
def selection_par_index_plage(fichier, index, borne_inf, borne_sup):  
  for i in range(borne_inf, borne_sup+1):
    num_page, position = index[i]
    lecture_tuple(fichier, num_page, position)



In [ ]:
t1 = time.time()
selection_par_index_plage("data.txt", index1, 17000, 17030)
print("done in", round(time.time() - t1, 2), "s")

17000,414817,1007979,305580,109061,13095,69233,----------------------------------------------------------------------------------------------------

17001,844669,1086043,290417,102384,74751,58097,----------------------------------------------------------------------------------------------------

17002,525922,739164,367591,6378,20563,58015,----------------------------------------------------------------------------------------------------

17003,518539,455181,450628,99435,132520,67633,----------------------------------------------------------------------------------------------------

17004,167710,622351,185656,102002,45013,73976,----------------------------------------------------------------------------------------------------

17005,1028090,882981,220190,81118,8561,68553,----------------------------------------------------------------------------------------------------

17006,1885654,1067928,122834,214087,145411,70459,----------------------------------------------------------------

#Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



In [ ]:
def addN(a0, N):
  page_dir = page_dir_name(DATA)
  num_page, position = index1[a0]

  lines=[]
  with open(page_dir + f"/page{num_page}", "r") as fpr:
    for i, line in enumerate(fpr):
      if(i==position-1):
        t = line.split(',')
        t[1]=str(int(t[1])+N)
        t=','.join(t)
        lines.append(t)
      else : 
        lines.append(line)

  with open(page_dir + f"/page{num_page}", "w") as fpw:
            fpw.write(''.join(lines))


In [ ]:
addN(844431,1)

##Modifier l'index en conséquence lorsque l'attibut modifié est indexé


In [ ]:
def updateIndex2(fichier, a0, N):
  num_page, position = index1[a0]
  print("tuple : ")
  t = selection_par_index(fichier, index1, a0)
  t = t.split(',')
  new_a1 = int(t[1])
  old_a1 = new_a1 - N
  #suppression 
  values = index2.pop(old_a1)
  values.pop(values.index((num_page,position)))
  index2[old_a1] = values
  #ajout
  if(new_a1 in index2):
    index2[new_a1] = index2[new_a1]+[(num_page, position)]
  else : 
    index2[new_a1] = [(num_page, position)]



In [ ]:
print("Avant : ")
t = selection_par_index(DATA, index1, 844431)
t = t.split(',')
old_a1 = int(t[1])
print(index2[old_a1])

updateIndex2(DATA, 844431, 1)
new_a1 = old_a1 + 1

print("Après : ")
print(index2[old_a1])
print(index2[new_a1])


Avant : 
844431,1528790,229042,24023,48545,123626,64316,----------------------------------------------------------------------------------------------------

[(4898, 770)]
tuple : 
844431,1528790,229042,24023,48545,123626,64316,----------------------------------------------------------------------------------------------------

Après : 
[(4898, 770), (188, 14)]
[(781, 708), (1209, 420), (4482, 794)]


# Persistence



## Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement


## Adapter en conséquence les opérations de modification de l'index